# install 
## docker compose
```yaml
version: '3.4'
services:
  typesense:
    image: typesense/typesense:0.25.0
    restart: on-failure
    ports:
      - "8108:8108"
    volumes:
      - ./typesense-data:/data
    command: '--data-dir /data --api-key=xyz --enable-cors'
```

### 启动脚本
```bash
mkdir $(pwd)/typesense-data
docker-compose up
```

### 关闭脚本
```bash
docker compose stop
```

## WSL
```bash
curl -O https://dl.typesense.org/releases/0.25.0/typesense-server-0.25.0-amd64.deb
sudo apt install ./typesense-server-0.25tart Typesense
sudo /usr/bin/./typesense-server --config=/etc/typesense/typesense-server
```

# Get Started

## install a client

In [4]:
!pip install typesense

In [3]:
!curl -O https://dl.typesense.org/datasets/books.jsonl.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  475k  100  475k    0     0   262k      0  0:00:01  0:00:01 --:--:--  262k


In [15]:
!gzip -dk books.jsonl.gz
!ls

gzip: books.jsonl already exists; do you wish to overwrite (y or n)? ^C
10-typesense-ug.ipynb  books.jsonl.gz  typesense-data
books.jsonl	       compose.yaml


## initializing the client
api_key= xyz 

port 8108

In [15]:
import typesense
client = typesense.Client({
  'api_key': 'xyz',
  'nodes': [{
    'host': 'localhost',
    'port': '8108',
    'protocol': 'http'
  }],
  'connection_timeout_seconds': 2
})

### creating a book collection

In [16]:
books_schema = {
  'name': 'books',
  'fields': [
    {'name': 'title', 'type': 'string' },
    {'name': 'authors', 'type': 'string[]', 'facet': True },

    {'name': 'publication_year', 'type': 'int32', 'facet': True },
    {'name': 'ratings_count', 'type': 'int32' },
    {'name': 'average_rating', 'type': 'float' }
  ],
  'default_sorting_field': 'ratings_count'
}

create_response = client.collections.create(books_schema)

ObjectAlreadyExists: [Errno 409] A collection with name `books` already exists.

In [10]:
with open("./books.jsonl") as jsonl_file:
    client.collections["books"].documents.import_(jsonl_file.read().encode("utf-8"))

In [14]:
search_parameters = {
  'q'         : 'harry potter',
  'query_by'  : 'title',
  'filter_by' : 'publication_year:<1998',
  'sort_by'   : 'ratings_count:desc'
}
client.collections['books'].documents.search(search_parameters)

{'facet_counts': [],
 'found': 1,
 'hits': [{'document': {'authors': ['J.K. Rowling', ' Mary GrandPré'],
    'average_rating': 4.44,
    'id': '2',
    'image_url': 'https://images.gr-assets.com/books/1474154022m/3.jpg',
    'publication_year': 1997,
    'ratings_count': 4602479,
    'title': "Harry Potter and the Philosopher's Stone"},
   'highlight': {'title': {'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}},
   'highlights': [{'field': 'title',
     'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}],
   'text_match': 1157451471441100921,
   'text_match_info': {'best_field_score': '2211897868288',
    'best_field_weight': 15,
    'fields_matched': 1,
    'score': '1157451471441100921',
    'tokens_matched': 2}}],
 'out_of': 9979,
 'page': 1,
 'request_params': {'collection_name': 'books',
  'per_page': 10,
  'q': 'harry potter'},
 's

In [6]:
search_parameters = {
  'q'         : 'harry potter',
  'query_by'  : 'title',
  'facet_by'  : 'authors',
  'sort_by'   : 'average_rating:desc'
}

res = client.collections['books'].documents.search(search_parameters)

In [7]:
from IPython.display import JSON

In [8]:
JSON(res, expanded=False)

<IPython.core.display.JSON object>

In [17]:
JSON(client.collections['books'].retrieve())

<IPython.core.display.JSON object>